<a href="https://colab.research.google.com/github/IsaacFigNewton/SMIED/blob/main/Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and config

In [19]:
from typing import List, Any, Dict, Tuple, Set
import math

import nltk
nltk.download('framenet_v17')
nltk.download('wordnet')
from nltk.corpus import framenet as fn
from nltk.corpus import wordnet as wn

from collections import defaultdict

import spacy
from spacy.tokens import Token, Doc
from spacy import displacy

max_samples = 2
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package framenet_v17 to /root/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Test different wn, fn library functionality

In [20]:
wn.synsets("running")[:max_samples]

[Synset('run.n.05'), Synset('run.n.07')]

In [21]:
pos = 'n'
cache = defaultdict(list)
for frame in fn.frames()[:max_samples]:
    print(frame.name)
    for i, (lu_name, lu_data) in enumerate(frame.lexUnit.items()):
      if i >= max_samples:
        break
      print(f"\t{lu_name}")
    print()

Abandonment
	abandon.v
	leave.v

Abounding_with
	teem.v
	crawling.a



# Get overlapping hypernym paths

In [ ]:
def overlapping_hypernym_paths(syn1, syn2) -> List[Any]:
    lchs = syn1.lowest_common_hypernyms(syn2)
    print("LCHs:", [lch.name() for lch in lchs])
    common_paths = []

    for p1 in syn1.hypernym_paths():
        for p2 in syn2.hypernym_paths():
            if any(lch in p1 for lch in lchs) and any(lch in p2 for lch in lchs):
              # truncate the paths until they've got one of the lchs
              while p1 and p2 and p1[0] not in lchs:
                  last_lch = p1[0]
                  p1 = p1[1:]
                  p2 = p2[1:]
              # get the shared lch path
              common_paths.append(p1[::-1] + p2[1:])

    return common_paths

In [ ]:
# Example:
cat = wn.synset('cat.n.01')
dog = wn.synset('dog.n.01')
print()
overlaps = overlapping_hypernym_paths(cat, dog)
for path in overlaps:
    print(" → ".join(s.name() for s in path))
print()
overlaps = overlapping_hypernym_paths(dog, cat)
for path in overlaps:
    print(" → ".join(s.name() for s in path))


LCHs: ['carnivore.n.01']
cat.n.01 → feline.n.01 → carnivore.n.01 → canine.n.02 → dog.n.01

LCHs: ['carnivore.n.01']
dog.n.01 → canine.n.02 → carnivore.n.01 → feline.n.01 → cat.n.01


# Get frame info from predicate token

## Get a dict of all dependency schemas matching a synset's frames/usage

In [3]:
import spacy
from spacy.tokens import Token

# Register a custom extension for original POS if you want to store it
if not Token.has_extension("orig_tag"):
    Token.set_extension("orig_tag", default=None)

@spacy.language.Language.component("custom_pos_modifier")
def custom_pos_modifier(doc):
    mapping = {"something": "NN", "someone": "NN"}  # Use suitable tag string
    for token in doc:
        # Save original tag
        token._.orig_tag = token.tag_

        if token.text.lower() not in mapping and token.pos_ == "NOUN":
            token.tag_ = "VB"  # Set fine-grained tag to verb
        # Else, you could override mapping if needed

    return doc

arg_schema_nlp = spacy.load("en_core_web_sm")
arg_schema_nlp.add_pipe("custom_pos_modifier", after="tagger")
print(arg_schema_nlp.pipe_names)

['tok2vec', 'tagger', 'custom_pos_modifier', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [4]:
def _flatten_tok_deps(tok: Token) -> Set[Token]:
    deps = set()
    for child in tok.children:
        deps.add(child)
        deps.update(_flatten_tok_deps(child))
    return deps

# TODO: retain original arg structure and add other syntactic/lexical info for better SRL
def _flattened_fn_arg_schema(doc: Doc|Token) -> Dict[str, str]:
      # used as fallback if FE parsing fails later on
      schema_atom_synset_maps = {
          "something": "entity.n.01",
          "someone": "causal_agent.n.01",
      }

      # if it's a token, just flatten its children and call that set a doc
      if isinstance(doc, Token):
          doc: Set[Token] = _flatten_tok_deps(doc)

      arg_schema = dict()
      for tok in doc:
          if tok.lower in schema_atom_synset_maps.keys():
              arg_schema[tok.dep_] = schema_atom_synset_maps[tok.lower]
          else:
              arg_schema[tok.dep_] = tok.lemma_
      return arg_schema

In [5]:
vect_order = ["subject", "object"]  #, "theme"]
arg_schema_dep_req_map = {
    "subject": {"nsubj", "nsubjpass"},
    "object": {"dobj", "obj", "pobj"},
    "theme": {"iobj"}
}

In [22]:
def _get_arg_schema_doc(lemma):
      # get the argument structure for this frame as a string
      f_str = lemma.frame_strings()[i]
      # parse f_str into an argument structure vector
      #   use tuple to make it hashable
      f_arg_structure = f_str.split(' ')

      # remove any extra arguments beyond subject, object, theme
      if f_arg_structure[-1] != f_arg_structure[-1].lower():
          f_arg_structure = f_arg_structure[:-1]

      # create a spacy doc for the argument structure template
      doc = arg_schema_nlp(' '.join(f_arg_structure))
      # # display dependency parse tree for debugging
      # displacy.render(doc, style='dep')
      return doc

In [23]:
def _get_dep_reqs(doc: Doc|Token) -> Tuple[bool, bool]:
      arg_schema_reqs = _flattened_fn_arg_schema(doc)

      # restructure arg_reqs based to only evaluate core dependencies
      #   TODO: add more relationships for finer-grained SRL
      return tuple([
          len(arg_schema_dep_req_map[k].intersection(set(arg_schema_reqs.keys()))) > 0
          for k in vect_order
      ])

In [24]:
def _get_f_id_arg_struct_dict(syn: wn.synset) -> Dict[int, Dict[str, bool]]:
      syn_frame_ids_strs: Dict[int, Doc] = dict()
      for lemma in syn.lemmas():

          # get all FrameNet frame IDs for this lemma
          for i, f_id in enumerate(lemma.frame_ids()):
              doc = _get_arg_schema_doc(lemma)

              syn_frame_ids_strs[f_id] = _get_dep_reqs(doc)

      return syn_frame_ids_strs

In [8]:
f_id_docs = _get_f_id_arg_struct_dict(wn.synset('spin.v.01'))

## Get Wordnet frames structures that match the structure of the dependency parse of the original predicate token

In [9]:
def _apply_frame_based_WSD_filter(pred_tok: spacy.tokens.Token) -> Tuple[Set[str], Tuple[bool, bool]]:
      # since wordnet frames only support a max of 2 args, even for ditransitive verbs,
      #   truncate to just subj, obj roles
      original_tok_dep_reqs = _get_dep_reqs(pred_tok)
      print(f"Original pred_tok dependency structure requirements: {original_tok_dep_reqs}")

      # get candidate WordNet synsets for the predicate (verbs)
      pred_lemma = pred_tok.lemma_.lower()
      # get a dict of synset names and synset objects for quick lookup
      pred_synsets = wn.synsets(pred_lemma, pos=wn.VERB)
      pred_synsets = {syn.name(): syn for syn in pred_synsets}

      # filter pred_synsets by their wordnet frames
      filtered_synsets: Set[str] = set()
      for s_name, syn in pred_synsets.items():
          for frame_id, arg_struct_tuple in _get_f_id_arg_struct_dict(syn).items():
              # ensure the proposed tuple matches the requirements of the original token
              if arg_struct_tuple == original_tok_dep_reqs:
                  filtered_synsets.add(s_name)

      return filtered_synsets, original_tok_dep_reqs

In [10]:
import pprint
test_doc = nlp("I struck the cobra with my axe.")
displacy.render(test_doc, style='dep')
pprint.pprint(_apply_frame_based_WSD_filter(test_doc[1]))

Original pred_tok dependency structure requirements: (True, True)
({'assume.v.05',
  'come_to.v.03',
  'fall_upon.v.01',
  'hit.v.12',
  'mint.v.01',
  'strickle.v.02',
  'strike.v.01',
  'strike.v.04',
  'strike.v.10',
  'strike.v.13',
  'strike.v.14',
  'strike.v.21'},
 (True, True))


## Test more wn, fn functionality

In [28]:
# # nouns don't have frames in wordnet
# syn = wn.synset('cat.n.01')
# for lemma in syn.lemmas():
#       doc = _get_arg_schema_doc(lemma)
#       displacy.render(doc, style='dep')

# Try getting framenet frame for wordnet sense using Universal Verb Index (UVI)